In [51]:
import chainer
import numpy
from seq2seq import Seq2seq
from seq2seq import load_npz_no_strict
from seq2seq import load_vocabulary
from seq2seq import UNK, EOS
from seq2seq import (stoch_translate, temp_translate,karpathy_translate, beam_search_translate)

In [52]:


# Globals
PATH_TO_MODEL = "../models"

# Some test sentences here
TEST_SENTENCES = ["The European Union promotes the German pickle.", 
                  "The German banks are pressing hard the Greek government.",
                  "The deputy prime minister of Spain crushed his population with a new tariff."]

SOURCE_VOCAB = "../raw_data/vocab.en"
TARGET_VOCAB = "../raw_data/vocab.es" 

# use -1 for CPU
GPU_ID = -1
UNITS = 1024
NUM_LAYERS = 3

# Load vocabularies for source and target languages
source_ids = load_vocabulary(SOURCE_VOCAB)
target_ids = load_vocabulary(TARGET_VOCAB)

# inverse dictionaries
target_words = {i: w for w, i in target_ids.items()}
source_words = {i: w for w, i in source_ids.items()}


# Init the model
model = Seq2seq(NUM_LAYERS, len(source_ids), len(target_ids), UNITS)

# load pretrained model
load_npz_no_strict("{}/model.npz".format(PATH_TO_MODEL), model)

# send model to GPU
if GPU_ID >= 0:
    chainer.cuda.get_device(GPU_ID).use()
    model.to_gpu(GPU_ID)
    
def preprocess(vocabulary, sentence_list):
    """ Translate a sentence to an ID sequence 
    Parameters:
    ----------
    vocabulary: source dict with word/id (key/value)
    sentence_list: list of strs
    
    """
    data = []
    for line in sentence_list:
        words = line.strip().split()
        array = numpy.array([vocabulary.get(w, UNK) for w in words], 'i')
        data.append(array)
    return data


In [53]:
# Preprocess Test sentences
sentence_list = preprocess(source_ids, TEST_SENTENCES)

print ("SENTENCE ", sentence_list)

SENTENCE  [array([  16,  265,  299, 4230,    2, 3286,    0], dtype=int32), array([  16, 3286, 3355,   31, 4631, 3508,    2, 5010,    0], dtype=int32), array([   16,  7101,  7309,  8231,     4,  1311, 29466,   120,   334,
          19,     9,   111,     0], dtype=int32)]


In [54]:
# 1-step argmax translation
for sentence, target in zip(sentence_list, TEST_SENTENCES):
    out = model.translate([sentence])
    print ("Target: ", target)
    result_sentence = ' '.join([target_words[y] for y in out[0].tolist()])
    print ("Result: ", result_sentence)
    

Target:  The European Union promotes the German pickle.
Result:  La Unión Europea promueve la <UNK> <UNK>
Target:  The German banks are pressing hard the Greek government.
Result:  Los bancos alemanes son <UNK> <UNK> de la <UNK> griega
Target:  The deputy prime minister of Spain crushed his population with a new tariff.
Result:  El jefe de la <UNK> <UNK> <UNK> con España con su población <UNK>


In [55]:
# Use stochastic sampling (typically used in language models to obtain diversity in samples)
NUM_SAMPLES_PER_SENTENCE = 4
for sentence, target in zip(sentence_list, TEST_SENTENCES):
    for i in range(NUM_SAMPLES_PER_SENTENCE):
        out = stoch_translate(model, [sentence])
        print ("Target: ", target)
        result_sentence = ' '.join([target_words[y] for y in out[0].tolist()])
        print ("Result: ", result_sentence)

Target:  The European Union promotes the German pickle.
Result:  La Unión Europea promueve la igualdad " <UNK> " .
Target:  The European Union promotes the German pickle.
Result:  La Unión Europea promueve el boicoteo " alemán "
Target:  The European Union promotes the German pickle.
Result:  La Unión Europea fomenta la base <UNK> alemana .
Target:  The European Union promotes the German pickle.
Result:  La Unión Europea promueve la firma de la <UNK> alemana .
Target:  The German banks are pressing hard the Greek government.
Result:  Los bancos alemanes respaldan la inversa los grecochipriotas
Target:  The German banks are pressing hard the Greek government.
Result:  La condición de alemán niega los “Estados <UNK> <UNK> Powell .
Target:  The German banks are pressing hard the Greek government.
Result:  Los respectivos bancos griegos reciben la Convención <UNK>
Target:  The German banks are pressing hard the Greek government.
Result:  <UNK> für <UNK> <UNK> den 00 sus recursos .
Target: 

In [48]:
# Use temperature to choose between argmax samples and stochastic sampling
NUM_SAMPLES_PER_SENTENCE = 4
TEMPERATURE = 0.1

for sentence, target in zip(sentence_list, TEST_SENTENCES):
    for i in range(NUM_SAMPLES_PER_SENTENCE):
        out = temp_translate(model, [sentence])
        print ("Target: ", target)
        result_sentence = ' '.join([target_words[y] for y in out[0].tolist()])
        print ("Result {}: {}".format(TEMPERATURE, result_sentence))
        



Target:  The European Union promotes the German pickle.
Result 0.1: La Unión Europea fomenta la <UNK> alemana
Target:  The European Union promotes the German pickle.
Result 0.1: La Unión Europea promueve la privatización alemana
Target:  The European Union promotes the German pickle.
Result 0.1: La Unión Europea promueve la <UNK> <UNK>
Target:  The European Union promotes the German pickle.
Result 0.1: La UE promueve la <UNK> " Alemania "
Target:  The German banks are pressing hard the Greek government.
Result 0.1: Los bancos griegos están gravemente siendo escépticos sin resolver .
Target:  The German banks are pressing hard the Greek government.
Result 0.1: Nouvelles Steiner es <UNK> <UNK> <UNK> para la <UNK>
Target:  The German banks are pressing hard the Greek government.
Result 0.1: Los bancos alemanes son <UNK> de una forma seria de Grecia
Target:  The German banks are pressing hard the Greek government.
Result 0.1: Los bancos alemanes son <UNK> en lo que se refiere a la honorabl

In [49]:
# Use temperature (Karpathy) to directly modify probs before sampling. 
# With low temperature the model is more convervative
# With higher temperature the output is less consistent (gramatically speaking)

NUM_SAMPLES_PER_SENTENCE
TEMPERATURE = 5.9

for sentence, target in zip(sentence_list, TEST_SENTENCES):
    for i in range(NUM_SAMPLES_PER_SENTENCE):
        out = karpathy_translate(model, [sentence])
        print ("Target: ", target)
        result_sentence = ' '.join([target_words[y] for y in out[0].tolist()])
        print ("Result {}: {}".format(TEMPERATURE, result_sentence))

Target:  The European Union promotes the German pickle.
Result 5.9: La Unión Europea promueve la <UNK> <UNK>
Target:  The European Union promotes the German pickle.
Result 5.9: La Unión Europea fomenta la <UNK> alemana .
Target:  The European Union promotes the German pickle.
Result 5.9: La Unión Europea promueve las <UNK> <UNK>
Target:  The European Union promotes the German pickle.
Result 5.9: La Unión Europea promueve la <UNK> <UNK>
Target:  The German banks are pressing hard the Greek government.
Result 5.9: Los bancos alemanes son particularmente difíciles de resolver sordos .
Target:  The German banks are pressing hard the Greek government.
Result 5.9: Los bancos alemanes son <UNK> críticos en el <UNK>
Target:  The German banks are pressing hard the Greek government.
Result 5.9: Los bancos alemanes son <UNK> sin el <UNK> de <UNK> .
Target:  The German banks are pressing hard the Greek government.
Result 5.9: Los bancos alemanes son débiles y han sido <UNK> por el hombre .
Target:

In [57]:
# Beam search. Mantain K best sequences at each step. It is a way to delay the decision of the next step
# However, similar samples are typically obtained. Used in translation with very large beams.
K_BEAM = 10
USE_UNK = False

for sentence, target in zip(sentence_list, TEST_SENTENCES):
    out, score = beam_search_translate(model, [sentence], K_BEAM, use_unk=USE_UNK)
    print ("Target: ", target)
    for i in range(len(out)):
        result_sentence = ' '.join([target_words[y] for y in out[i]])
        print ("Results {}: {}".format(score[i], result_sentence))
    

/media/hcerezo/7ab36923-d61a-42bf-b119-e1820eb4fa22/nmt/myseq2seq/seq2seq.py:299: RuntimeWarning: invalid value encountered in log
  cand_scores = hyp_scores[:, None] - model.xp.log(probs_data[:live_k, :])


Target:  The European Union promotes the German pickle.
Results 6.001535415649414: La Unión Europea fomenta el boicoteo alemán <EOS>
Results 6.042962074279785: La Unión Europea promueve el boicoteo alemán <EOS>
Results 6.442540168762207: La Unión Europea promueve la privatización alemana <EOS>
Results 7.401790142059326: La Unión Europea promueve el boicoteo de Alemania <EOS>
Results 7.685765266418457: La Unión Europea fomenta el boicoteo de Alemania <EOS>
Results 8.061744689941406: La Unión Europea promueve la privatización alemana . <EOS>
Results 8.833370208740234: La Unión Europea promueve la privatización de Alemania <EOS>
Results 9.2477388381958: La Unión Europea promueve el boicoteo de Alemania . <EOS>
Results 9.69877815246582: La Unión Europea fomenta el boicoteo de Alemania . <EOS>
Results 10.125044822692871: La Unión Europea promueve el boicoteo de Alemania ; <EOS>
Target:  The German banks are pressing hard the Greek government.
Results 10.088885307312012: Los bancos alemanes 